In [1]:
# Cell 1: Initial setup, connecting to Google Drive, installing libraries, and checking GPU availability.
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Instalar y actualizar las librerías
print("\nInstalando y actualizando librerías...")
!pip install --upgrade -q mne pytorch-lightning timm
print("✅ Librerías listas.")

# Step 3: Prueba explícita de control de la GPU
import torch
print("\n--- INICIANDO PRUEBA DE CONTROL DE GPU ---")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
    try:
        tensor_grande = torch.randn(1024, 1024, 512, device=device) # Asignar 2GB
        memoria_asignada = torch.cuda.memory_allocated(0) / 1024**3
        print(f"✅ ¡Éxito! Memoria asignada activamente: {memoria_asignada:.2f} GB")
        del tensor_grande
        torch.cuda.empty_cache()
        print("✅ Memoria liberada correctamente.")
        print("--- PRUEBA DE CONTROL DE GPU COMPLETADA EXITOSAMENTE ---")
    except Exception as e:
        print(f"❌ ¡ERROR DURANTE LA PRUEBA! No se pudo asignar memoria a la GPU: {e}")
else:
    print("❌ ¡ERROR! No se detectó ninguna GPU en este entorno de ejecución.")

Mounted at /content/drive

Instalando y actualizando librerías...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 73.2 MB/s eta 0:00:00
✅ Librerías listas.

--- INICIANDO PRUEBA DE CONTROL DE GPU ---
✅ GPU detectada: NVIDIA A100-SXM4-40GB
✅ ¡Éxito! Memoria asignada activamente: 2.00 GB
✅ Memoria liberada correctamente.
--- PRUEBA DE CONTROL DE GPU COMPLETADA EXITOSAMENTE ---


In [ ]:
# ==============================================================================
# 1. SETUP AND DATA PREPARATION
# ==============================================================================
from google.colab import drive
from pathlib import Path
import os

# Mount Google Drive to access the dataset
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)
print("✅ Google Drive mounted.")

# --- MODIFIED: Conditionally copy data to local disk for stability and speed ---
# This check prevents re-copying files on every run.
local_shhs_dir = Path('/content/local_shhs_data')

if local_shhs_dir.exists() and any(local_shhs_dir.glob('**/*.parquet')):
    print(f"\n✅ Data already exists in {local_shhs_dir}. Skipping file copy.")
else:
    print(f"\nLocal data not found. Copying dataset from Google Drive...")

    # Define source directories on Google Drive
    gdrive_shhs1_dir = Path('/content/drive/MyDrive/shhs1_processed')
    gdrive_shhs2_dir = Path('/content/drive/MyDrive/shhs2_processed')

    # Define local destination directories in the Colab environment
    local_shhs1_dir = local_shhs_dir / 'shhs1'
    local_shhs2_dir = local_shhs_dir / 'shhs2'

    # Create the local directories
    os.makedirs(local_shhs1_dir, exist_ok=True)
    os.makedirs(local_shhs2_dir, exist_ok=True)

    # Use shell commands to copy the first 100 files from each directory
    get_ipython().system(f"ls -p '{gdrive_shhs1_dir}' | grep -v / | head -n 100 | xargs -I % cp '{gdrive_shhs1_dir}/%' '{local_shhs1_dir}/'")
    get_ipython().system(f"ls -p '{gdrive_shhs2_dir}' | grep -v / | head -n 100 | xargs -I % cp '{gdrive_shhs2_dir}/%' '{local_shhs2_dir}/'")

    print("✅ Data copying complete.")

# ==============================================================================
# 2. DEPENDENCY INSTALLATION
# ==============================================================================
print("\nEnsuring PyTorch Lightning and other libraries are installed...")
!pip install --upgrade -q pytorch-lightning timm "pandas==2.2.2" "pyarrow==19.0.0"
print("✅ Installation check complete.")

# ==============================================================================
# 3. IMPORTS AND INITIAL CONFIGURATION
# ==============================================================================
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassPrecision, MulticlassRecall
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.set_float32_matmul_precision('medium')
print("✅ Libraries imported and configuration set.")

# ==============================================================================
# 4. MODEL ARCHITECTURE DEFINITION (SINGLE-MODEL)
# ==============================================================================
def get_model(model_name='convnext_tiny', num_classes=5, pretrained=True):
    """Creates a ConvNeXT Tiny model adapted for sleep stage classification."""
    if model_name == 'convnext_tiny':
        model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)
        original_conv = model.stem[0]
        new_first_conv = nn.Conv2d(1, original_conv.out_channels, kernel_size=original_conv.kernel_size, stride=original_conv.stride, padding=original_conv.padding, bias=(original_conv.bias is not None))
        with torch.no_grad():
            if original_conv.weight.shape[1] == 3:
                new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
        model.stem[0] = new_first_conv
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        print(f"✅ ConvNeXT Tiny model created.")
    else:
        raise ValueError(f"Model '{model_name}' not supported in this script.")
    return model

print("✅ `get_model` function defined for ConvNeXT Tiny.")

# ==============================================================================
# 5. PYTORCH LIGHTNING MODULE
# ==============================================================================
class SleepStageClassifierLightning(pl.LightningModule):
    def __init__(self, model_name, learning_rate=1e-5, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_model(model_name=self.hparams.model_name, num_classes=5, pretrained=True)
        self.train_accuracy = MulticlassAccuracy(num_classes=5)
        self.val_accuracy = MulticlassAccuracy(num_classes=5)
        self.loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', self.train_accuracy(y_pred_logits, y_true), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', self.val_accuracy(y_pred_logits, y_true), on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = {'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3), 'monitor': 'val_loss'}
        return [optimizer], [scheduler]

print("✅ `SleepStageClassifierLightning` module defined.")

# ==============================================================================
# 6. CUSTOM DATASET DEFINITION
# ==============================================================================
class CombinedDataset(Dataset):
    def __init__(self, file_paths_chunk):
        print(f"Initializing dataset with {len(file_paths_chunk)} files...")
        self.file_paths = file_paths_chunk
        self.epochs_per_file = [pd.read_parquet(f, columns=['label'])['label'].isin([0, 1, 2, 3, 4]).sum() for f in self.file_paths]
        self.cumulative_epochs = np.cumsum(self.epochs_per_file)
        self.total_epochs = self.cumulative_epochs[-1] if self.cumulative_epochs.size > 0 else 0
        self._cache = {}
        print(f"✅ Dataset initialized. Total valid epochs: {self.total_epochs}")

    def __len__(self):
        return self.total_epochs

    def __getitem__(self, idx):
        file_idx = np.searchsorted(self.cumulative_epochs, idx, side='right')
        local_idx = idx - (self.cumulative_epochs[file_idx - 1] if file_idx > 0 else 0)
        file_path = self.file_paths[file_idx]
        if file_path not in self._cache:
            df = pd.read_parquet(file_path)
            self._cache[file_path] = df[df['label'].isin([0, 1, 2, 3, 4])].reset_index(drop=True)
        row = self._cache[file_path].iloc[local_idx]
        label = np.int64(row['label'])
        spectrogram_flat = row.drop('label').values.astype(np.float32)
        mean, std = spectrogram_flat.mean(), spectrogram_flat.std()
        spectrogram_normalized = (spectrogram_flat - mean) / (std + 1e-6)
        spectrogram_2d = spectrogram_normalized.reshape(1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.tensor(label)

print("✅ `CombinedDataset` class defined.")

# ==============================================================================
# 7. PERFORMANCE REPORTING FUNCTION
# ==============================================================================
def generate_performance_report(model_checkpoint_path, dataloader, device):
    """Loads the best model and generates a detailed classification report."""
    print("\n" + "="*80)
    print("Generating Final Performance Metrics on the Validation Set...")
    model = SleepStageClassifierLightning.load_from_checkpoint(model_checkpoint_path)
    model.to(device)
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in dataloader:
            logits = model(x.to(device))
            all_preds.append(torch.argmax(logits, dim=1).cpu())
            all_labels.append(y.cpu())
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    num_classes = 5
    metrics = {
        "Precision": MulticlassPrecision(num_classes=num_classes, average=None),
        "Recall": MulticlassRecall(num_classes=num_classes, average=None),
        "F1-Score": MulticlassF1Score(num_classes=num_classes, average=None)
    }
    results = {name: metric(all_preds, all_labels) for name, metric in metrics.items()}
    accuracy = MulticlassAccuracy(num_classes=num_classes, average='micro')(all_preds, all_labels)

    stage_map = {0: "Wake", 1: "N1", 2: "N2", 3: "N3", 4: "REM"}
    print("\n--- Sleep Stage Classification Report (Best Model) ---")
    print(f"{'Stage':<10} | {'Precision':<10} | {'Recall':<10} | {'F1-Score':<10}")
    print("-" * 50)
    for i in range(num_classes):
        print(f"{stage_map[i]:<10} | {results['Precision'][i]:<10.4f} | {results['Recall'][i]:<10.4f} | {results['F1-Score'][i]:<10.4f}")
    print("-" * 50)
    print(f"\nOverall Accuracy: {accuracy.item():.4f}")
    print("="*80 + "\n")

print("✅ `generate_performance_report` function defined.")

# ==============================================================================
# 8. TRAINING EXECUTION
# ==============================================================================
print("\n--- Starting Model Training ---")

# --- General Parameters ---
MODEL_TO_TEST = 'convnext_tiny'
EPOCHS = 40
BATCH_SIZE = 256
NUM_WORKERS = os.cpu_count()
CLASS_WEIGHTS = [0.7, 3.5, 0.5, 1.5, 1.2]
LEARNING_RATE = 5e-5

# --- Paths point to the local Colab directories ---
shhs1_processed_dir_base = Path('/content/local_shhs_data/shhs1')
shhs2_processed_dir_base = Path('/content/local_shhs_data/shhs2')

specific_shhs_file_paths = list(shhs1_processed_dir_base.glob('*.parquet')) + list(shhs2_processed_dir_base.glob('*.parquet'))

# --- Main Experiment ---
if not specific_shhs_file_paths:
     print("\nERROR: No .parquet files found. Aborting.")
else:
    print(f"\nFound {len(specific_shhs_file_paths)} files for training.")
    full_dataset = CombinedDataset(specific_shhs_file_paths)

    if len(full_dataset) > 1:
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
        print(f"Dataset split: {len(train_dataset)} training, {len(val_dataset)} validation.")

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=bool(NUM_WORKERS))
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=bool(NUM_WORKERS))

        print(f"\n{'='*20} STARTING EXPERIMENT FOR MODEL: {MODEL_TO_TEST.upper()} {'='*20}")
        model = SleepStageClassifierLightning(MODEL_TO_TEST, LEARNING_RATE, CLASS_WEIGHTS)

        experiment_name = f"{MODEL_TO_TEST}_local_test"
        csv_logger = CSVLogger("/content/drive/MyDrive/sleep_logs/", name=experiment_name)

        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath='/content/drive/MyDrive/final_model_checkpoint/',
            filename=f"sleep-stage-{experiment_name}-{{epoch:02d}}-{{val_loss:.4f}}",
            save_top_k=1, mode='min'
        )
        early_stop_callback = EarlyStopping(monitor='val_loss', patience=7, verbose=True, mode='min')

        trainer = pl.Trainer(
            max_epochs=EPOCHS, accelerator="gpu", devices=1, logger=csv_logger,
            callbacks=[checkpoint_callback, early_stop_callback],
            precision="bf16-mixed", gradient_clip_val=1.0
        )

        print(f"🚀 Starting model training for {MODEL_TO_TEST.upper()}...")
        trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
        print(f"✅ Training complete for {MODEL_TO_TEST.upper()}!")

        if checkpoint_callback.best_model_path:
            print(f"Best model saved at: {checkpoint_callback.best_model_path}")
            generate_performance_report(checkpoint_callback.best_model_path, val_loader, model.device)
        else:
            print("No checkpoint was saved. Skipping performance report.")
        print(f"{'='*20} FINISHED EXPERIMENT FOR MODEL: {MODEL_TO_TEST.upper()} {'='*20}")
    else:
        print("Dataset is too small to split. Aborting.")

print("\n--- Model Training Complete ---")

Mounting Google Drive...
Mounted at /content/drive
✅ Google Drive mounted.

Local data not found. Copying dataset from Google Drive...
^C
^C
✅ Data copying complete.

Ensuring PyTorch Lightning and other libraries are installed...
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/provider.py", line 162, in get_preference
    requested_order: Union[int, float] = self._user_requested[identifier]
                                         ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'typing-extensions'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
          